In [124]:
import pandas as pd
import numpy as np
import gmaps
import requests
from config import g_key

In [125]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [139]:
city_data_df = pd.read_csv('data/WeatherPy_challenge.csv')
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Min Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (Inches),Snow (Inches),Date
0,0,Timmins,CA,48.47,-81.33,53.60,53.60,93,90,8.05,light rain,0.0,0.0,2020-05-29 15:51:19
1,1,Katsuura,JP,35.13,140.30,68.00,66.00,88,75,6.93,broken clouds,0.0,0.0,2020-05-29 15:51:19
2,2,Ushuaia,AR,-54.80,-68.30,44.60,44.60,56,75,9.17,broken clouds,0.0,0.0,2020-05-29 15:51:19
3,3,Daru,PG,-9.08,143.21,78.94,78.94,88,98,14.74,light rain,0.5,0.0,2020-05-29 15:51:19
4,4,Ketchikan,US,55.34,-131.65,55.40,51.80,71,40,13.87,scattered clouds,0.0,0.0,2020-05-29 15:50:04


In [140]:
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
raining = input("Do you want it to be raining? (yes/no) ")
snowing = input("Do you want it to be snowing? (yes/no) ")

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90
Do you want it to be raining? (yes/no) no
Do you want it to be snowing? (yes/no) no


In [141]:
# Filter the dataset to find the cities that fit the temp criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

# Filter for rain
if raining == 'yes':
    preferred_cities_df = preferred_cities_df.loc[(city_data_df['Rain (Inches)'] > 0)]
else:
    preferred_cities_df = preferred_cities_df.loc[(city_data_df['Rain (Inches)'] == 0)]

# Filter for snow
if snowing == 'yes':
    preferred_cities_df = preferred_cities_df.loc[(city_data_df['Snow (Inches)'] > 0)]
else:
    preferred_cities_df = preferred_cities_df.loc[(city_data_df['Snow (Inches)'] == 0)]
    

In [142]:
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Min Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (Inches),Snow (Inches),Date
7,7,Georgetown,MY,5.41,100.34,84.20,82.00,83,20,8.05,few clouds,0.0,0.0,2020-05-29 15:51:19
12,12,Ladario,BR,-19.00,-57.60,82.40,82.40,48,82,9.17,broken clouds,0.0,0.0,2020-05-29 15:51:20
13,13,Saint-Philippe,RE,-21.36,55.77,73.40,61.00,78,38,6.93,scattered clouds,0.0,0.0,2020-05-29 15:51:20
14,14,Saint George,US,37.10,-113.58,84.99,78.80,22,1,2.21,clear sky,0.0,0.0,2020-05-29 15:50:55
15,15,Hilo,US,19.73,-155.09,71.60,66.99,78,90,5.82,overcast clouds,0.0,0.0,2020-05-29 15:51:20


In [143]:
preferred_cities_df['City'].count()

233

In [144]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
7,Georgetown,MY,84.20,few clouds,5.41,100.34,
12,Ladario,BR,82.40,broken clouds,-19.00,-57.60,
13,Saint-Philippe,RE,73.40,scattered clouds,-21.36,55.77,
14,Saint George,US,84.99,clear sky,37.10,-113.58,
15,Hilo,US,71.60,overcast clouds,19.73,-155.09,
20,Rikitea,PF,74.91,clear sky,-23.12,-134.97,
23,Bethel,US,73.99,mist,41.37,-73.41,
24,Salalah,OM,77.00,light rain,17.02,54.09,
28,Sao Joao Da Barra,BR,77.00,few clouds,-21.64,-41.05,
37,Dunmore,US,82.40,broken clouds,41.42,-75.63,


In [145]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [146]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [151]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
7,Georgetown,MY,84.20,few clouds,5.41,100.34,Cititel Penang
12,Ladario,BR,82.40,broken clouds,-19.00,-57.60,Hotel Fazenda 4 cantos
13,Saint-Philippe,RE,73.40,scattered clouds,-21.36,55.77,"Chambres d'hôte ""La Trinité"""
14,Saint George,US,84.99,clear sky,37.10,-113.58,Best Western Plus Abbey Inn
15,Hilo,US,71.60,overcast clouds,19.73,-155.09,Hilo Hawaiian Hotel


In [148]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.1)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, 
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)


# Call the figure to plot the data.
fig



Figure(layout=FigureLayout(height='420px'))

In [149]:
# Save DF as CSV

hotel_df.to_csv('data/WeatherPy_vacation.csv', index_label="City_ID")